In [1]:
#import numpy as np
import model
import pickle
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from model.entity_type import EntityType as et
from math import ceil

### player_view

In [11]:
player_view = pickle.load(open("player_view.dat", "rb"))

In [15]:
player_view.entity_properties[et.BUILDER_UNIT].population_use

1

In [16]:
player_view.entity_properties[et.BUILDER_UNIT].population_provide

0

### houses

In [5]:
map_size = 80
entities = pickle.load(open("entities_1.dat", "rb"))

In [7]:
city = [
    [11, 11, et.HOUSE],
    [11, 16, et.HOUSE],
    [16, 11, et.HOUSE],
    [6, 11, et.HOUSE],
]

### damaged

In [72]:
my_id = 1
actions = {}
repairers_per_target = 2

In [50]:
mask = (entities['player_id'] == my_id) & ( \
       (entities['entity_type'] == et.BUILDER_BASE) | \
       (entities['entity_type'] == et.MELEE_BASE) | \
       (entities['entity_type'] == et.RANGED_BASE) | \
       (entities['entity_type'] == et.TURRET) | \
       (entities['entity_type'] == et.HOUSE)) & \
       (entities['health_gap'] > 0)
damaged = pd.DataFrame(entities[mask], columns=['entity_id','x','y','health_gap'])
damaged.columns = ['entity_id','dx','dy','health_gap']
damaged.sort_values(['health_gap'], ascending=False, inplace = True)
damaged

,entity_id,dx,dy,health_gap
17,7,15,5,135


In [73]:
busy = []
mask = (entities['player_id'] == my_id) & (entities['entity_type'] == et.BUILDER_UNIT)
for building_id, dx, dy, health_gap in damaged.values:
    repairers = pd.DataFrame(entities[mask], columns=['entity_id', 'x', 'y'])
    repairers.columns = ['entity_id', 'rx', 'ry']    
    repairers['distance']=np.hypot(repairers.rx-dx,repairers.ry-dy)
    repairers.sort_values(['distance'], ascending=True, inplace = True)
    repairers_count = 0
    for repairer_id, rx, ry, distance in repairers.values:
        if not repairer_id in busy:
            m, b, a, r = None, None, None, None
            target = model.vec2_int.Vec2Int(dx, dy)
            r = model.repair_action.RepairAction(target=building_id)
            m = model.move_action.MoveAction(target, True, True)
            actions[repairer_id] = model.EntityAction(m, b, a, r)
            busy.append(repairer_id)
            repairers_count += 1
            if repairers_count >= repairers_per_target:
                break

### house

In [103]:
from math import sqrt

In [119]:
debug_params = pickle.load(open("debug_params_1.dat", "rb"))

In [120]:
my_id, entities, actions, initial_cost, resource = debug_params

In [121]:
initial_cost * 2 < resource

True

In [131]:
#if initial_cost * 2 < resource:
# get exists houses
mask = (entities['player_id'] == my_id) & (entities['entity_type'] == et.HOUSE)
houses = pd.DataFrame(entities[mask], columns=['x', 'y'])
# check for empty spaces
build_spaces = []
quad_limit = int(sqrt(len(houses))+4)*4
for hx in range(1, quad_limit, 5):
    for hy in range(1, quad_limit, 5):
        if len(houses[(houses.x == hx) & (houses.y == hy)]) == 0:
            build_spaces.append([hx, hy])
build_spaces = pd.DataFrame(build_spaces, columns=['hx', 'hy'])

In [138]:
4 % (2 * 2)

0

In [132]:
quad_limit

24

In [123]:
len(build_spaces)

1

In [112]:
ban_houses = pd.DataFrame([[6, 6]], columns=['nx', 'ny'])

In [115]:
hx, hy = 6, 6
mask = (ban_houses.nx == hx) & (ban_houses.ny == hy)
len(ban_houses[mask])

1

In [128]:
if len(build_spaces):        
    build_spaces['zero_distance'] = np.hypot(build_spaces.hx, build_spaces.hy)
    build_spaces.sort_values(['zero_distance'], ascending = True, inplace = True)
    tasks = 0
    busy = []
    mask = (entities['player_id'] == my_id) & (entities['entity_type'] == et.BUILDER_UNIT)
    for hx, hy, zero_distance in build_spaces.values:
        if len(busy) > 0:
            break
        hx = int(hx)
        hy = int(hy)
        if len(ban_houses[(ban_houses.nx == hx) & (ban_houses.ny == hy)]) > 0:
            continue
        builders = pd.DataFrame(entities[mask], columns=['entity_id', 'x', 'y'])
        builders.columns = ['entity_id', 'bx', 'by']
        builders['distance'] = np.hypot(builders.bx - hx, builders.by - hy)
        builders.sort_values(['distance'], ascending=True, inplace=True)
        for builder_id, bx, by, distance in builders.values:
            builder_id = int(builder_id)
            if not builder_id in busy:
                m, b, a, r = None, None, None, None
                target = model.vec2_int.Vec2Int(hx, hy)
                b = model.build_action.BuildAction(et.HOUSE, model.vec2_int.Vec2Int(hx, hy))
                m = model.move_action.MoveAction(model.vec2_int.Vec2Int(hx - 1, hy), True, True)
                actions[builder_id] = model.EntityAction(m, b, a, r)
                busy.append(builder_id)
                print('house', builder_id, hx, hy)
                break
                #repairers_count += 1
                #if repairers_count >= self.repairers_per_target:
                #    break

house 4001 1 16


In [186]:
enemy_centroids = pd.DataFrame([[5,  21] ], columns=['x', 'y'])

In [157]:
sight_range = 10

In [176]:
unit_len, unit_counter , ceil(unit_len / counter_len), 

(18, 9, 6)

In [187]:
actions = {}
mask = (entities['player_id'] == my_id) & \
       (
               (entities['entity_type'] == et.MELEE_UNIT) |
               (entities['entity_type'] == et.RANGED_UNIT)
       )
units = pd.DataFrame(entities[mask], columns=['entity_id','x','y'])
units.columns = ['entity_id', 'ux', 'uy']
i = 0
for cx, cy in enemy_centroids.values:
    units['d'+str(i)] = np.hypot(units.ux - cx, units.uy - cy)
    i += 1

busy = []
#i = 0
unit_len = len(units)
counter_len = len(enemy_centroids)
unit_counter = 0
current_centroid = 0
units.sort_values(['d'+str(current_centroid)], ascending=True, inplace=True)
for ex, ey in enemy_centroids.values:
    #if unit_counter > unit_len / counter_len:
    #x,y = self.enemy_centroids.values[0]
    target = model.vec2_int.Vec2Int(ex, ey)

    m, b, a, r = None, None, None, None
    m = model.move_action.MoveAction(target, True, True)
    a = model.attack_action.AttackAction(
        target=None,
        auto_attack=model.auto_attack.AutoAttack(
            pathfind_range=sight_range,
            valid_targets=[]
        )
    )
    for entity_id in units.entity_id:
        if not entity_id in busy:
            actions[entity_id] = model.EntityAction(m, b, a, r)
            busy.append(entity_id)
            unit_counter += 1            
            if unit_counter >= unit_len / counter_len:
                unit_counter = 0
                current_centroid += 1
                units.sort_values(['d'+str(current_centroid-1)], ascending=True, inplace=True)
                break
    #i += 1

In [188]:
actions

{3979: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3964: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3811: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3794: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3469: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3550: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3538: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3743: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3662: EntityAction(MoveAction(Vec2Int(5,21),True,True),None,AttackAction(None,AutoAttack(10,[])),None),
 3552: EntityAction(MoveAction(Vec2Int(5,21),True,True)

In [ ]:
    def repairers_action(self, timer_start, timer_end, my_id, entities, actions):
        if self.debug:
            timer_start['repairers_action'] = datetime.datetime.now()

        mask = (entities['player_id'] == my_id) & ( \
                    (entities['entity_type'] == et.BUILDER_BASE) | \
                    (entities['entity_type'] == et.MELEE_BASE) | \
                    (entities['entity_type'] == et.RANGED_BASE) | \
                    (entities['entity_type'] == et.TURRET) | \
                    (entities['entity_type'] == et.HOUSE)) & \
               (entities['health_gap'] > 0)
        damaged = pd.DataFrame(entities[mask], columns=['entity_id', 'x', 'y', 'health_gap'])
        damaged.columns = ['entity_id', 'dx', 'dy', 'health_gap']
        damaged.sort_values(['health_gap'], ascending=False, inplace=True)
        if len(damaged):
            busy = []
            mask = (entities['player_id'] == my_id) & (entities['entity_type'] == et.BUILDER_UNIT)
            for building_id, dx, dy, health_gap in damaged.values:
                repairers = pd.DataFrame(entities[mask], columns=['entity_id', 'x', 'y'])
                repairers.columns = ['entity_id', 'rx', 'ry']
                repairers['distance'] = np.hypot(repairers.rx - dx, repairers.ry - dy)
                repairers.sort_values(['distance'], ascending=True, inplace=True)
                repairers_count = 0
                for repairer_id, rx, ry, distance in repairers.values:
                    repairer_id = int(repairer_id)
                    if not repairer_id in busy:
                        m, b, a, r = None, None, None, None
                        target = model.vec2_int.Vec2Int(dx, dy)
                        r = model.repair_action.RepairAction(target=building_id)
                        m = model.move_action.MoveAction(target, True, True)
                        actions[repairer_id] = model.EntityAction(m, b, a, r)
                        busy.append(repairer_id)
                        repairers_count += 1
                        if repairers_count >= self.repairers_per_target:
                            break

        if self.debug:
            timer_end['repairers_action'] = datetime.datetime.now()

        return actions